In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
url='https://www.billboard.com/charts/billboard-global-200/'
response = requests.get(url)

In [4]:
response.status_code

200

In [5]:
soup= BeautifulSoup(response.content)

In [6]:
song_title= soup.select('h3.a-no-trucate')[0::2]
art= soup.select('span.a-no-trucate')[0::2]

In [7]:
tp_song= [i.get_text(strip=True) for i in song_title]
artits= [i.get_text(strip=True) for i in art]

In [8]:
tp_song

['Ordinary',
 'Die With A Smile',
 'Birds Of A Feather',
 'Beautiful Things',
 'Shake It To The Max (Fly)',
 'Love Me Not',
 'What I Want',
 'Espresso',
 'Triple Lavada',
 'Undressed',
 'Pink Pony Club',
 'Wildflower',
 'That’s So True',
 'Mystical Magical',
 'Just In Case',
 'No Tiene Sentido',
 'Azizam',
 'Nokia',
 'Marlboro Rojo',
 'No One Noticed',
 'Gnarly',
 'Anxiety',
 'Sailor Song',
 'Si Antes Te Hubiera Conocido',
 'All The Stars',
 'Party 4 U',
 'Creep',
 'Dreams',
 'Blue Strips',
 'Taste',
 'Fame Is A Gun',
 'Mutt',
 'EOO',
 'Yellow',
 'Perfect',
 'Gata Only',
 'Please Please Please',
 'I Got Better',
 'Sweater Weather',
 'Viva La Vida',
 'Nuevayol',
 'Snooze',
 'Cruel Summer',
 'Capaz',
 'Luna Bala',
 'Who',
 'As It Was',
 'Me Jalo',
 'Blinding Lights',
 'Take On Me',
 'Move',
 'Te Queria Ver',
 "Sorry I'm Here For Someone Else",
 'Chihiro',
 'See You Again',
 'Blue',
 'Voy A Llevarte Pa PR',
 'I Adore You',
 'Numb',
 'Montagem Tomada',
 'Por Esos Ojos',
 'Love Somebody',
 

In [9]:
print(len(tp_song))
print(len(artits))


100
100


In [10]:
song_dict={'Songs':tp_song,'Artist': artits}
song_df=pd.DataFrame(song_dict)
song_df

,Songs,Artist
0,Ordinary,Alex Warren
1,Die With A Smile,Lady Gaga & Bruno Mars
2,Birds Of A Feather,Billie Eilish
3,Beautiful Things,Benson Boone
4,Shake It To The Max (Fly),"MOLIY, Silent Addy, Skillibeng & Shenseea"
...,...,...
95,Until I Found You,Stephen Sanchez
96,Worst Way,Riley Green
97,The Chain,Fleetwood Mac
98,Cliche,mgk


In [11]:
import spotipy
import pandas as pd
import json
from spotipy.oauth2 import SpotifyClientCredentials
import  config

In [19]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.CLIENT_ID,
                                                           client_secret= config.CLIENT_SECRET
                                                           ))


In [20]:
def get_track_id(songs, artist):
    query = f"{songs}{artist}"
    result = sp.search(q=query, type='track', limit=1)
    tracks = result.get('tracks', {}).get('items', [])
    if tracks:
        return tracks[0]['id']
    return  None
                         

In [21]:

song_df['track_id'] = song_df.apply(lambda row: get_track_id(row['Songs'], row['Artist']), axis=1)


In [22]:
song_df

,Songs,Artist,track_id
0,Ordinary,Alex Warren,6qqrTXSdwiJaq8SO0X2lSe
1,Die With A Smile,Lady Gaga & Bruno Mars,2QmXQ2s0B0CLRQcMLEHif9
2,Birds Of A Feather,Billie Eilish,6dOtVTDdiauQNBQEDOtlAB
3,Beautiful Things,Benson Boone,4CDXYCzPMXTAi6nuyixVnM
4,Shake It To The Max (Fly),"MOLIY, Silent Addy, Skillibeng & Shenseea",0QCIpQV3twfqo9kh0t8Zza
...,...,...,...
95,Until I Found You,Stephen Sanchez,2RkZ5LkEzeHGRsmDqKwmaJ
96,Worst Way,Riley Green,7GX5flRQZVHRAGd6B4TmDO
97,The Chain,Fleetwood Mac,1IMa0Il4w988MwZCUcISix
98,Cliche,mgk,3aD33slymVwdEcdbKUBnDu


In [24]:
track_id=[]
track_id=song_df['track_id']

In [25]:
track_id

0     6qqrTXSdwiJaq8SO0X2lSe
1     2QmXQ2s0B0CLRQcMLEHif9
2     6dOtVTDdiauQNBQEDOtlAB
3     4CDXYCzPMXTAi6nuyixVnM
4     0QCIpQV3twfqo9kh0t8Zza
               ...          
95    2RkZ5LkEzeHGRsmDqKwmaJ
96    7GX5flRQZVHRAGd6B4TmDO
97    1IMa0Il4w988MwZCUcISix
98    3aD33slymVwdEcdbKUBnDu
99    4tkiLZ2Q7SuaeSiSi56rnJ
Name: track_id, Length: 100, dtype: object

In [ ]:
import os
os.makedirs("playlist", exist_ok=True)
csv_path = os.path.join("playlist", "top_100_billboard.csv")
song_df.to_csv(csv_path, index=False)

